## Exercises - Map Reduce Functions
Here are the same exercises which you have solved before. Try to solve these using out-of-the-box Map Reduce APIs.

* Get number of COMPLETE orders placed by each customer
* Get total number of PENDING or PENDING_PAYMENT orders for the month of 2014 January.
* Get outstanding amount for each month considering orders with status PAYMENT_REVIEW, PENDING, PENDING_PAYMENT and PROCESSING.

### Details of Data

Here are the details about the orders data which you can leverage to take care of these exercises.
* Location: `/data/retail_db/orders/part-00000`
* Each record is line separated or line delimited.
* Attribute in each record is comma separated.
* Here are the columns in the orders data set.
  * order_id
  * order_date
  * order_customer_id
  * order_status

In [ ]:
# Get the details about file
!ls -ltr /data/retail_db/orders/part-00000

In [ ]:
# Get first five lines from the file
!head -5 /data/retail_db/orders/part-00000

In [ ]:
# Get number of lines from the file
# We can use linux command wc with -l
!wc -l /data/retail_db/orders/part-00000

Here are the details about the order_items data which you can leverage to take care of these exercises.
* Location: `/data/retail_db/order_items/part-00000`
* Each record is line separated or line delimited.
* Attribute in each record is comma separated.
* Here are the columns in the order_items data set.
  * order_item_id
  * order_item_order_id
  * order_item_product_id
  * order_item_quantity
  * order_item_subtotal
  * order_item_product_price

In [ ]:
# Get the details about file
!ls -ltr /data/retail_db/order_items/part-00000

In [ ]:
# Get first five lines from the file
!head -5 /data/retail_db/order_items/part-00000

In [ ]:
# Get number of lines from the file
# We can use linux command wc with -l
!wc -l /data/retail_db/order_items/part-00000

### Exercise 1 - Read data from file
Before getting into problem statement, develop the code to read the file into list of elements.
* We should be able to use this function to read any file with text data using line as record delimiter.

In [2]:
# Update the logic here
def get_list_from_file(file_path):
    data_list = open(file_path).read().splitlines()
    return data_list

* Run below cells to validate the function
* You should see 68883 records as part of the output for the cell with `len(orders)` below.
* You should see 172198 records as part of the output for the cell with `len(order_items)` below.

In [3]:
orders = get_list_from_file('D:\\BIGDATA-LEARN\\data-engineering-spark-main\\data\\retail_db\\orders\\part-00000')

In [4]:
orders[:5]

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [5]:
len(orders)

68883

In [6]:
order_items = get_list_from_file('D:\\BIGDATA-LEARN\\data-engineering-spark-main\\data\\retail_db\\order_items\\part-00000')

In [8]:
order_items[:5]

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

In [9]:
len(order_items)

172198

### Exercise 2 - Complete Order Count by Customer

Get number of COMPLETE orders placed by each customer. Develop a function which read the orders data and get us complete order count by each customer using **order_customer_id**.
* The function should take the complete order list as argument and return count of complete orders by customer. The function should return **dict** type object.
* The order is said to be complete if the **order_status** is **COMPLETE**.
* You can review structure of the data under **Details of Data** section in this notebook.

In [18]:
# Update the logic here
# You can use `filter` and `len` to solve this problem.
def get_complete_order_count_by_customer(orders):
    order_filter = filter(lambda x:x.split(',')[3]=='COMPLETE', orders)
    order_map = sorted(list(map(lambda x:(x.split(',')[2],1), order_filter)), key=lambda x:x[0])
    order_count_by_customer = {}
    for i,j in order_map:
        order_count_by_customer[i]=order_count_by_customer.get(i,j)+1
    return order_count_by_customer

* Run below cell to validate the function. You should get **22899** as output.

In [19]:
orders = get_list_from_file('D:\\BIGDATA-LEARN\\data-engineering-spark-main\\data\\retail_db\\orders\\part-00000')

In [20]:
complete_order_count_by_customer = get_complete_order_count_by_customer(orders)

In [21]:
# This should return dict
type(complete_order_count_by_customer)

dict

In [22]:
# This should return 10538
len(complete_order_count_by_customer)

10538

* Run below cell to preview the data.

In [23]:
for e in sorted(complete_order_count_by_customer.items())[:5]:
    print(e)

('1', 2)
('10', 3)
('100', 4)
('1000', 5)
('10000', 3)


### Exercise 3 - Pending Order Count

Get total number of PENDING or PENDING_PAYMENT orders for the month of 2014 January. Develop a function which read the orders data and get us pending order count.
* The function should take the complete order list as argument and return count of pending orders.
* The order is said to be complete if the status is **PENDING** or **PENDING_PAYMENT**. We should only consider the orders placed in the month of 2014 January.
* The second element in each comma separated record gives us the date
* The 4th or last element in each comma separated record gives us the order status.
* This problem can also be solved using `filter` and `len`.

In [26]:
# Update the logic here
# You need to use myFilter and myReduce for this
def get_pending_order_count(orders):
    order_filter = filter(lambda x:x.split(',')[3] in ['PENDING','PENDING_PAYMENT'] and x.split(',')[1].startswith('2014-01'), orders)
    return len(list(order_filter))

* Run below cell to validate your function. You should get **1969** as output.

In [27]:
get_pending_order_count(orders)

1969

* You can also validate results using simple linux scripts.

In [ ]:
!egrep -w '(PENDING|PENDING_PAYMENT)' /data/retail_db/orders/part-00000|grep 2014-01|wc -l

### Exercise 4 - Get Outstanding Revenue

Get outstanding amount for each month considering orders with status PAYMENT_REVIEW, PENDING, PENDING_PAYMENT and PROCESSING.
* You can use **order_item_subtotal** to compute the outstanding amount.
* Here are the instructions for the solution.
  * Create a list of tuples by name `pending_orders` for pending orders using `filter`. 
  * Use `map` function to convert the `pending_orders` to list of tuples so that we can join with order items data. Each tuple in the list should contain order_id and hard coded value 1. You can overwrite the `pending_orders` with this new list of tuples.
  * Create a list of tuples by name `order_item` for order_items in which each tuple contain `order_id` and `order_item_subtotal`. You can use `map` function for this.
  * Use the `myJoin` function which is developed as part of the previous section or module. It is provided as part of next cell.
  * Create a new list by name `order_item_subtotals` by invoking `myJoin` using `pending_orders` and `order_items`. 
  * `order_item_subtotals` is of type `list` of tuples where first element in each tuple is order_id and second element is a nested tuple which contain elements related to `pending_orders` and `order_items`..
  * We can then use `map` function to get `order_item_subtotal` and then use `sum` to get the outstanding revenue.
  * Make sure to use `round` to round off to 2 decimals.
* Review **Details of Data** section to get more details of columns.

In [94]:
def myJoin(c1, c2):
    c1_dict = dict(c1) # dict with first element as key and second element as value
    results = [] # Initializing empty list
    for c2_item in c2: 
        if c2_item[0] in c1_dict:
            results.append((c1_dict[c2_item[0]], c2_item[1]))
    return results

In [95]:
pending_orders = filter(lambda x:x.split(',')[3] in ['PENDING','PENDING_PAYMENT','PROCESSING','PAYMENT_REVIEW'], orders)
pending_orders = list(map(lambda x:(x.split(',')[0], x.split(',')[1][:7]), pending_orders))
pending_orders[:5]

[('2', '2013-07'),
 ('8', '2013-07'),
 ('9', '2013-07'),
 ('10', '2013-07'),
 ('11', '2013-07')]

In [96]:
order_item = list(map(lambda x:(x.split(',')[1],x.split(',')[-1]), order_items))
order_item[:5]

[('1', '299.98'),
 ('2', '199.99'),
 ('2', '50.0'),
 ('2', '129.99'),
 ('4', '24.99')]

In [97]:
from itertools import groupby, starmap

In [98]:
date_merge = sorted(myJoin(pending_orders, order_item), key=lambda x:x[0])

In [99]:
order_grouped_date = groupby(date_merge, lambda x:x[0])

In [100]:
month_revenue = starmap(lambda k,v: (k, round(sum(list(map(lambda x:float(x[1]), v))), 2)),
                        order_grouped_date)
print(list(month_revenue)[:5])

[('2013-07', 248116.42), ('2013-08', 887216.53), ('2013-09', 888593.5), ('2013-10', 805947.91), ('2013-11', 992644.85)]
